In [41]:
import torch
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [231]:
raw_data = pd.read_csv("dsex-val2.csv")
raw_data["Date"] = pd.to_datetime(raw_data["Date"], dayfirst=True)
raw_data.head()

output = pd.DataFrame()

current_month = 0


for row_idx in range(1, len(raw_data)):
    current_row = raw_data.iloc[:row_idx,: ]
    i = 0
    if current_row["Date"].dt.month != current_month:
        current_month = current_row["Date"].dt.month
        
    break;
        

0    3
Name: Date, dtype: int64


In [175]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
class DSEX_Dataset(Dataset):
    def __init__(self, csv_file_name, day_window, transform=None):
        self.raw_data = pd.read_csv(csv_file_name)
        self.raw_data["Date"] = pd.to_datetime(self.raw_data["Date"], dayfirst=True)
        self.raw_data_tensor = torch.tensor(self.raw_data.iloc[:, 1:].values)
        
        data_mean = torch.mean(self.raw_data_tensor, dim=0)
        data_std = torch.std(self.raw_data_tensor, dim=0)
        
        self.raw_data_tensor = (self.raw_data_tensor - data_mean) / data_std
        
        self.dates = self.raw_data.iloc[:, 0]
        self.transform = transform;
        self.day_window = day_window;
        self.date_data = self.date_to_onehot(self.dates)
    
    def __len__(self):
        return len(self.raw_data) - self.day_window*2;
    
    def __getitem__(self, idx):
        start = idx;
        end = idx+self.day_window
        x = self.raw_data_tensor[start:end, :] 
        y = self.raw_data_tensor[end:(end + self.day_window), 1].unsqueeze(1)
        y_date = self.date_data[end:(end + self.day_window), :]
        
        return (x,y,y_date)
    
    def date_to_onehot(self, dates):
        one_hot = np.zeros((len(dates), 20)) # 12 for months and 7 for days
#         one_hot[:, dates.dt.day]
        
        one_hot[np.arange(len(dates)), dates.dt.dayofweek.values] = 1
        one_hot[:, 7:][np.arange(len(dates)), dates.dt.month] = 1
        return one_hot
        
         
    
dsex_dataset = DSEX_Dataset("dsex-val2.csv", 5, transform=None);
    
dsex_loader = DataLoader(dsex_dataset, batch_size=len(dsex_dataset), shuffle=False)
print(dsex_dataset[0])


(tensor([[1.5480, 1.0737],
        [0.5071, 1.0942],
        [1.0091, 1.0626],
        [0.4619, 0.9547],
        [0.6828, 0.9111]], dtype=torch.float64), tensor([[0.9429],
        [0.8147],
        [0.7800],
        [0.7983],
        [0.7499]], dtype=torch.float64), array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]]))


In [55]:
input_features = 2; # [ADTV, DSEX]
seq_len = 5; # stock market open week
batch_size = 50;
hidden_size = 256;
num_layers = 1;

inputs = torch.randn(batch_size, 1, input_features)
h1 = torch.randn(num_layers, batch_size, hidden_size)
c1 = torch.randn(num_layers, batch_size, hidden_size)

lstm = nn.LSTM(input_features, hidden_size, num_layers=num_layers, bias=True, batch_first=True)


seq_output = torch.empty(50, 5, input_features)
for i in range(5):
    output, (h1, c1) = lstm(inputs, (h1, c1))
    torch.cat((seq_output, output), dim=0)

print(output.shape)
print(h1.shape)



RuntimeError: Sizes of tensors must match except in dimension 0. Got 5 and 1 in dimension 1 (The offending index is 1)

In [203]:
# 5 12 [0 0 0 1 0 0, 0 0 0 0 0 0 0 ]

"""
    EdgyEncoder
    Input size: [ADTV, DSEX]
    Seq Len = day window
    hidden_size: 256
    num_layers: 124
    output_size: 128
    
    EdgyDecoder
    Input size: 128 (EdgyEncoder Output)
    Seq Len = day window
    hidden_size: 256
    num_layers: 124
    output_size: 1 (index of the next day)
    

"""
import torch.nn as nn


class EdgyEncoder(nn.Module):
    def __init__(self, input_features, hidden_size, num_layers):
        super().__init__();
        self.input_features = input_features 
        self.hidden_size = hidden_size;
        self.num_layers = num_layers;
        self.encoder = nn.LSTM(input_features, hidden_size, num_layers, bias=True, batch_first=True);
        
    def forward(self, x, hidden):
        x, hidden = self.encoder(x, hidden);
        return x, hidden;
    
class EdgyDecoder(nn.Module):
    def __init__(self, input_features, hidden_size, num_layers):
        super().__init__();
        self.input_features = input_features 
        self.hidden_size = hidden_size;
        self.num_layers = num_layers;
        self.decoder = nn.LSTM(input_features, hidden_size, num_layers, bias=True, batch_first=True);
        
        
    def forward(self, x, hidden):
        x, hidden = self.decoder(x, hidden);
        return x, hidden;
    
class FCNet(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__();
        self.fc1 = nn.Linear(input_features, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, output_features)
        self.relu = nn.ReLU();
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        return x;
    
class Edgy(nn.Module):
    def __init__(self, enc_input_features, dec_input_features, dec_hidden_size, fc_output_features, num_layers=1):
        super().__init__();
        self.enc_input_features = enc_input_features;
        self.dec_input_features = dec_input_features;
        self.dec_hidden_size = dec_hidden_size;
        self.fc_output_features = fc_output_features;
        self.num_layers = num_layers;
        self.edgy_encoder = EdgyEncoder(enc_input_features, dec_input_features, num_layers=num_layers)
        self.edgy_decoder = EdgyDecoder(dec_input_features, dec_hidden_size, num_layers=num_layers)
        self.fcnet = FCNet(dec_hidden_size, fc_output_features)

    
    def forward(self, x):
        h1 = torch.randn(self.num_layers, x.shape[0], self.dec_input_features)
        c1 = torch.randn(self.num_layers, x.shape[0], self.dec_input_features)
        
        enc_output, (h1, c1) = self.edgy_encoder(x.float(), (h1, c1))
        dec_output, (h1, c1) = self.edgy_decoder(enc_output, (h1, c1))
        
        #fcinput = torch.cat((dec_output, fc_y_date), dim=2)
        pred = self.fcnet(dec_output.float())
        
        return pred
        
        
    

In [210]:
"""
    EdgyEncoder
    Input size: [ADTV, DSEX]
    Seq Len = day window
    hidden_size: 256
    num_layers: 124
    output_size: 128
    
    EdgyDecoder
    Input size: 128 (EdgyEncoder Output)
    Seq Len = day window
    hidden_size: 256
    num_layers: 124
    output_size: 1 (index of the next day)
    

"""

import torch.optim as optim
from torch.nn import MSELoss
from tqdm import trange
import matplotlib.pyplot as plt


input_features = 2; # [ADTV, DSEX]
input_features_decoder = 256
seq_len = 5; # stock market open week
batch_size = 28;
hidden_size = 256;
num_layers = 3;
output_size = 1;

# edgy_encoder = EdgyEncoder(input_features, seq_len, hidden_size, num_layers=num_layers)
# edgy_decoder = EdgyDecoder(input_features_decoder, seq_len, hidden_size, num_layers=num_layers)
# fcnet = FCNet(hidden_size, output_size)

model = Edgy(input_features, hidden_size, hidden_size, output_size, num_layers=num_layers)


edgy_optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.MSELoss();

#ForwardPass
dsex_loader = DataLoader(dsex_dataset, batch_size=batch_size, shuffle=False)


dayofweek = ["sunday", "monday", "tuesday", "wednesday", "thursday"]

epochs = 50
t = trange(epochs, desc="current-loss", leave=True)
for epoch in t:
    losses = 0;
    n_batch = 0;
    for i, data in enumerate(dsex_loader):
        n_batch += 1
        enc_x, dec_y, fc_y_date = data;
        
        current_batch_size = enc_x.shape[0]
        

        if i == 0:
            test_pred = model(enc_x)
            print("prediction: \n")
            print(test_pred[0].detach().numpy())
            print("target: \n")
            target = dec_y
            print(target[0].numpy())
            
            print(criterion(test_pred, target).item())
            
        
        pred = model(enc_x)
        
        
        edgy_optimizer.zero_grad()
        
        loss = criterion(pred.float(), dec_y.float())
        losses += loss.item()
        loss.backward()
        
        edgy_optimizer.step()
    
    epoch_loss = losses/n_batch
    t.set_description(desc="current-loss: %f" %epoch_loss)
    losses = 0;
    n_batch = 0;
#     print(model.fcnet.fc2.weight[0].grad)
        

    
    
    

current-loss:   0%|                                                                             | 0/50 [00:00<?, ?it/s]

prediction: 

[[0.14642957]
 [0.14679559]
 [0.14717653]
 [0.14737763]
 [0.14737755]]
target: 

[[0.94291067]
 [0.81470634]
 [0.78004082]
 [0.79828618]
 [0.74989137]]
0.867754226771541


current-loss: 0.933207:   2%|█▏                                                         | 1/50 [00:03<03:00,  3.68s/it]

prediction: 

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
target: 

[[0.94291067]
 [0.81470634]
 [0.78004082]
 [0.79828618]
 [0.74989137]]
1.1513539934194612


current-loss: 1.000789:   4%|██▎                                                        | 2/50 [00:07<02:55,  3.65s/it]

...

current-loss: 1.000808:  96%|███████████████████████████████████████████████████████▋  | 48/50 [02:42<00:06,  3.41s/it]

prediction: 

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
target: 

[[0.94291067]
 [0.81470634]
 [0.78004082]
 [0.79828618]
 [0.74989137]]
1.152522554279337


current-loss: 1.000804:  98%|████████████████████████████████████████████████████████▊ | 49/50 [02:46<00:03,  3.39s/it]

prediction: 

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
target: 

[[0.94291067]
 [0.81470634]
 [0.78004082]
 [0.79828618]
 [0.74989137]]
1.152522554279337


current-loss: 1.000804: 100%|██████████████████████████████████████████████████████████| 50/50 [02:49<00:00,  3.40s/it]


In [192]:
print(pred[0])

tensor([[0.4678],
        [1.1061],
        [1.1024],
        [0.9786],
        [0.9268]], grad_fn=<SelectBackward>)


In [159]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split

In [170]:
x,y,y_date = next(iter(DataLoader(dsex_dataset, batch_size=len(dsex_dataset), shuffle=False)))

x, y = x.numpy(), y.numpy()


train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.33) 




model = Sequential()
model.add(LSTM(256, input_shape=(5, 2), return_sequences=True))
model.add(LSTM(256, input_shape=(5, 256), return_sequences=True))
model.add(LSTM(256, input_shape=(5, 256)))
model.add(Dense(1))
print(model.summary())
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_x, train_y, epochs=600, batch_size=64, verbose=2)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 5, 256)            265216    
_________________________________________________________________
lstm_19 (LSTM)               (None, 5, 256)            525312    
_________________________________________________________________
lstm_20 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 257       
Total params: 1,316,097
Trainable params: 1,316,097
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/600
21/21 - 4s - loss: 0.2432
Epoch 2/600
21/21 - 1s - loss: 0.0341
Epoch 3/600
21/21 - 1s - loss: 0.0281
Epoch 4/600
21/21 - 1s - loss: 0.0246
Epoch 5/600
21/21 - 1s - loss: 0.0240
Epoch 6/600
21/21 - 1s - loss: 0.0240
Epoch 7/

KeyboardInterrupt: 

KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_13/BiasAdd:0', description="created by layer 'dense_13'")


0.013094279915094376